In [2]:
from collections import Counter
import math
import pandas as pd
#import dataset
df=pd.read_csv('PlayTennis.csv')
n=len(df)   # Number of obs
print(len(df))
# print dataframe
df

14


,PlayTennis,Outlook,Temperature,Humidity,Wind
0,No,Sunny,Hot,High,Weak
1,No,Sunny,Hot,High,Strong
2,Yes,Overcast,Hot,High,Weak
3,Yes,Rain,Mild,High,Weak
4,Yes,Rain,Cool,Normal,Weak
5,No,Rain,Cool,Normal,Strong
6,Yes,Overcast,Cool,Normal,Strong
7,No,Sunny,Mild,High,Weak
8,Yes,Sunny,Cool,Normal,Weak
9,Yes,Rain,Mild,Normal,Weak


In [8]:
#Entropy
def entropy(ls):   # passing the list
    counts=Counter(x for x in ls)    # Comprehension
    total=len(ls)
    probs=[x/total for x in counts.values()]    #Comprehension
    E=sum(-p*math.log(p,2) for p in probs)      #comprehension
    return E

In [9]:
counts = Counter(x for x in df['PlayTennis'])
print(counts)

#check
entropy(df['PlayTennis'])

Counter({'Yes': 9, 'No': 5})


0.9402859586706309

In [11]:
def inf_gain(data,a,target):     # pass the subset
    df_split=data.groupby(a)
    n=len(data)
    df_agg=df_split.agg({target:[entropy,                                
                        lambda x : len(x)/n]})[target]
    df_agg.columns=['Entropy','Proportion']              # for new heading for entropy and proportions
    new_E=sum(df_agg['Entropy']*df_agg['Proportion'])
    old_E=entropy(df[target])
    return old_E - new_E

In [30]:
#Just for understanding
dfs=df.groupby('Outlook')    
#df_agg=dfs.agg('sum')['PlayTennis']
#print(df_agg)
for a,v in dfs:
    print(a)
    print(v)

Overcast
   PlayTennis   Outlook Temperature Humidity    Wind
2         Yes  Overcast         Hot     High    Weak
6         Yes  Overcast        Cool   Normal  Strong
11        Yes  Overcast        Mild     High  Strong
12        Yes  Overcast         Hot   Normal    Weak
Rain
   PlayTennis Outlook Temperature Humidity    Wind
3         Yes    Rain        Mild     High    Weak
4         Yes    Rain        Cool   Normal    Weak
5          No    Rain        Cool   Normal  Strong
9         Yes    Rain        Mild   Normal    Weak
13         No    Rain        Mild     High  Strong
Sunny
   PlayTennis Outlook Temperature Humidity    Wind
0          No   Sunny         Hot     High    Weak
1          No   Sunny         Hot     High  Strong
7          No   Sunny        Mild     High    Weak
8         Yes   Sunny        Cool   Normal    Weak
10        Yes   Sunny        Mild   Normal  Strong


In [31]:
#check 
dfs.agg('sum')['PlayTennis']

Outlook
Overcast     YesYesYesYes
Rain        YesYesNoYesNo
Sunny        NoNoNoYesYes
Name: PlayTennis, dtype: object

In [14]:
# For Checking
ig=inf_gain(df,'Outlook','PlayTennis')
print(ig)
ig=inf_gain(df,'Temperature','PlayTennis')
print(ig)
ig=inf_gain(df,'Wind','PlayTennis')
print(ig)
ig=inf_gain(df,'Humidity','PlayTennis')
print(ig)

0.2467498197744391
0.029222565658954647
0.04812703040826927
0.15183550136234136


In [20]:
#id3
def id3(df,target,attr,def_class=None,def_atrr='S'):
    pn = Counter(x for x in df[target])             #Comprehension -Total no.of positive and negative examples
    print(f'\n ** {pn}**')
    
    if len(pn)==1:                      #return yes or no    (only yes or only no)
        return next(iter(pn))
    elif df.empty or (not attr):
        return def_class
    else:                                              #actual algorithm starts here
        def_class=max(pn,key=pn.get) 
        print(f'Default class {def_class}')
        gains={}
        for a in attr:
            gains[a]=inf_gain(df,a,target)
            print(f'Inf gain on {a}:{gains[a]:0.04f}')
            
        best=max(gains,key=gains.get)
        print(f'Max gain {best}')
            
        #tree
        tree={best:{}}  #initialize tree with best at root
        attr.remove(best)     # remove root node from attribute list
            
        for av, data in df.groupby(best):
            #print(av)
            #print(data)
            subtree = id3(data,target,attr,def_class,best)    # subset
            tree[best][av]=subtree
            print(f'best {best} {av}')
            print(tree)
    return tree

In [38]:
#check
c=Counter(x for x in df['PlayTennis'])
print(c)
next(iter(c))

Counter({'Yes': 9, 'No': 5})


'No'

In [21]:
attr=list(df.columns)              #display 'PlayTennis' column in output
attr.remove('PlayTennis')     # Remove PlayTennis column
print('Predicting attributes',attr)

tree=id3(df,'PlayTennis',attr)

Predicting attributes ['Outlook', 'Temperature', 'Humidity', 'Wind']

 ** Counter({'Yes': 9, 'No': 5})**
Default class Yes
Inf gain on Outlook:0.2467
Inf gain on Temperature:0.0292
Inf gain on Humidity:0.1518
Inf gain on Wind:0.0481
Max gain Outlook

 ** Counter({'Yes': 4})**
best Outlook Overcast
{'Outlook': {'Overcast': 'Yes'}}

 ** Counter({'Yes': 3, 'No': 2})**
Default class Yes
Inf gain on Temperature:-0.0107
Inf gain on Humidity:-0.0107
Inf gain on Wind:0.9403
Max gain Wind

 ** Counter({'No': 2})**
best Wind Strong
{'Wind': {'Strong': 'No'}}

 ** Counter({'Yes': 3})**
best Wind Weak
{'Wind': {'Strong': 'No', 'Weak': 'Yes'}}
best Outlook Rain
{'Outlook': {'Overcast': 'Yes', 'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}}}

 ** Counter({'No': 3, 'Yes': 2})**
Default class No
Inf gain on Temperature:0.5403
Inf gain on Humidity:0.9403
Max gain Humidity

 ** Counter({'No': 3})**
best Humidity High
{'Humidity': {'High': 'No'}}

 ** Counter({'Yes': 2})**
best Humidity Normal
{'Humid

In [ ]:
#tree=id3(df,'PlayTennis',attr)

In [22]:
# Resultant Tree
from pprint import pprint
print('Tree \n')
pprint(tree)
ba=next(iter(tree))
print('Best attribute',ba)
print('Tree keys: \n',tree[ba].keys())

Tree 

{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}
Best attribute Outlook
Tree keys: 
 dict_keys(['Overcast', 'Rain', 'Sunny'])


<h1> Revision </h1>


In [4]:
df.head()
df.tail()
df.columns
df.values
df.values.tolist()
df.loc[:,'Humidity']     #loc - access using column name
#list of columns
df.loc[2:5,['Outlook','Humidity']]

#iloc
df.iloc[:,1]
df.iloc[:,1:]
df.iloc[:,-1]
df.iloc[:,[2,3]]

,Temperature,Humidity
0,Hot,High
1,Hot,High
2,Hot,High
3,Mild,High
4,Cool,Normal
5,Cool,Normal
6,Cool,Normal
7,Mild,High
8,Cool,Normal
9,Mild,Normal


In [5]:
df['Outlook'].unique()

array(['Sunny', 'Overcast', 'Rain'], dtype=object)

In [17]:
# Counter
c=Counter(x for x in df['PlayTennis'])
print(c)
print(next(iter(c)))

Counter({'Yes': 9, 'No': 5})
No


In [7]:
#groupby
ss=df.groupby('Outlook')
for k,v in ss:
    print('Attribute value',k)
    print(v)

Attribute value Overcast
   PlayTennis   Outlook Temperature Humidity    Wind
2         Yes  Overcast         Hot     High    Weak
6         Yes  Overcast        Cool   Normal  Strong
11        Yes  Overcast        Mild     High  Strong
12        Yes  Overcast         Hot   Normal    Weak
Attribute value Rain
   PlayTennis Outlook Temperature Humidity    Wind
3         Yes    Rain        Mild     High    Weak
4         Yes    Rain        Cool   Normal    Weak
5          No    Rain        Cool   Normal  Strong
9         Yes    Rain        Mild   Normal    Weak
13         No    Rain        Mild     High  Strong
Attribute value Sunny
   PlayTennis Outlook Temperature Humidity    Wind
0          No   Sunny         Hot     High    Weak
1          No   Sunny         Hot     High  Strong
7          No   Sunny        Mild     High    Weak
8         Yes   Sunny        Cool   Normal    Weak
10        Yes   Sunny        Mild   Normal  Strong
